## Preprocessing

In [4]:
#load data
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("nsmc")

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])


Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807 [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/6.33M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


In [5]:
#simplify

import re

docs = dataset['train']['document'] + dataset['test']['document']
label = dataset['train']['label'] + dataset['test']['label']

processed_docs = [re.sub("[\s]+", " ", re.sub("[^가-힣a-zA-Z0-9]", " ", doc)) for doc in docs]

In [6]:
processed_docs[20:50]

['나름 심오한 뜻도 있는 듯 그냥 학생이 선생과 놀아나는 영화는 절대 아님',
 '보면서 웃지 않는 건 불가능하다',
 '재미없다 지루하고 같은 음식 영화인데도 바베트의 만찬하고 넘 차이남 바베트의 만찬은 이야기도 있고 음식 보는재미도 있는데 이건 볼게없다 음식도 별로 안나오고 핀란드 풍경이라도 구경할랫는데 그것도 별로 안나옴 ',
 '절대 평범한 영화가 아닌 수작이라는걸 말씀드립니다 ',
 '주제는 좋은데 중반부터 지루하다',
 '다 짤랐을꺼야 그래서 납득할 수 없었던거야 그럴꺼야 꼭 그랬던걸꺼야 ',
 'kl2g 고추를 털어버려야 할텐데',
 '카밀라벨 발연기',
 '재밋는뎅',
 '센스있는 연출력 탁월한 캐스팅 90년대의 향수 그래서 9점 ',
 '엄포스의 위력을 다시 한번 깨닫게 해준 적 남 꽃검사님도 연기 정말 좋았어요 완전 명품드라마 ',
 '졸쓰레기 진부하고말도안됌 아 시간아까워',
 '재밌는데 별점이 왜이리 낮은고',
 '1 라도 기대했던 내가 죄인입니다 죄인입니다 ',
 '아직도 이 드라마는 내인생의 최고 ',
 '패션에 대한 열정 안나 윈투어 ',
 '키이라 나이틀리가 연기하고자 했던건 대체 정신장애일까 틱장애일까',
 '허허 원작가 정신나간 유령이라 재미있겠네요 ',
 '포스터는 있어보이는데 관객은 114명이네',
 '이 영화가 왜 이렇게 저평가 받는지 모르겠다',
 '단순하면서 은은한 매력의 영화',
 ' 다 알바생인가 내용도 없고 무서운거도 없고 웃긴거도 하나도 없음 완전 별싱거운 영화 내 시간 넘 아까움 완전 낚임',
 '오게두어라 서리한이 굶주렸다 ',
 '정말 맘에 들어요 그래서 또 보고싶은데 또 보는 방법이 없네 ',
 '윤제문이라는 멋진 배우를 발견하게 됐어요 소소한 일탈이 잔잔한 미소를 머금게 합니다 음악은 조금 아쉽네요 8점 주고 싶은데 평점 올리고 싶어 10점 줄게요 ',
 '평점에속지마시길시간낭비 돈낭비임',
 '리얼리티가 뛰어나긴 한데 큰 공감은 안간다 이민기캐릭터는 정신의학상 분노조절장애 초기 증상일거다 툭하면 사람

In [7]:
from konlpy.tag import Okt

def tokenize(docs):

  okt = Okt()

  for doc in docs:
    tokenlist = okt.pos(doc)
    temp = []
    for w in tokenlist:
      if w[1] in ['Noun', 'Verb', 'Adjective', 'Adverb', 'Exclamation', 'Foreign', 'Alpha', 'Number', 'Unknown']:
        temp.append(w[0])
    res.append(temp)
  return res


ModuleNotFoundError: ignored

In [ ]:
# tokenize

# from konlpy.tag import Mecab

# mecab = Mecab()

# res = []

# for doc in docs:
#   tokenlist = mecab.pos(doc)
#   for w in tokenlist:
#     if w[1] in ['NNG', 'NNP', 'NNB', 'NNBC', 'NP' 'VV', 'VA', 'IC', 'SN', 'SL', 'MAG', 'MAJ']:
#       res.append(w[1])

from konlpy.tag import Okt
import pickle

okt = Okt()

res = []

if not os.path.isfile('senspos.pickle'):
  for doc in docs:
    tokenlist = okt.pos(doc)
    temp = []
    for w in tokenlist:
      if w[1] in ['Noun', 'Verb', 'Adjective', 'Adverb', 'Exclamation', 'Foreign', 'Alpha', 'Number', 'Unknown']:
        temp.append(w[0])
    res.append(temp)

  with open("senspos.pickle", 'wb') as f:
    pickle.dump(res, f)
  
else:
  with open("senspos.pickle", 'rb') as f:
    res = pickle.load(f)
  


In [ ]:
# padding
maxlen = max(len(x) for x in res)
padded_sens = []
for i in range(len(res)):
  sen = res[i]
  temp = sen + [" <PAD/>"] * (maxlen - len(sen))
  padded_sens.append(temp)


In [ ]:
# vocab to index

import nltk
tokens = [t for d in padded_sens for t in d]
text = nltk.Text(tokens, name='NSMC')
word_count = text.vocab()
vocabulary_inv = [x[0] for x in word_count.most_common()]
vocabulary = {x:i for i, x in enumerate(vocabulary_inv)}


In [ ]:
# padded sens to index sens

x = np.array([[vocabulary[w] for w in sentence] for sentence in padded_sens])
y = np.array(label)

In [ ]:
# word 2 vec
from gensim.models import word2vec
import numpy as np

w2v = word2vec.Word2Vec.load("../data/w2v/ko.bin")

w2v = {k: w2v[w] if w in w2v else np.random.uniform(-0.25, 0.25, w2v.vector_size) for w, k in vocabulary.items()}


In [ ]:
# args setting

embedding_dim = 200
filter_sizes = (3, 4, 5)
num_filters = 100
dropout = 0.5
hidden_dims = 100

batch_size = 
num_epochs = 10
min_word_count = 1
context = 10

In [ ]:
# make dataset to use at training


X_train = x[:len(train_df)]
y_train = y[:len(train_df)]
X_test = x[len(train_df):]
y_test = y[len(train_df):]

X_train = np.stack([np.stack([w2v[w] for w in sen]) for sen in X_train])
X_test = np.stack([np.stack([w2v[w] for w in sen]) for sen in X_test])

In [ ]:
# setting model and train

from tensorflow import keras

inp = keras.layers.Input(shape=(X_test.shape[1] ,embedding_dim))

m = inp
m = keras.layers.Dropout(dropout)(m)

# build conv block
conv_blocks = []
for s in filter_sizes:
  conv = keras.layers.Conv1D(filters=num_filters,
                         kernel_size=s,
                         padding="valid",
                         activation="relu",
                         strides=1)(m)
  conv = keras.layers.MaxPooling1D(pool_size=2)(conv)
  conv = keras.layers.Flatten()(conv)
  conv_blocks.append(conv)
m = keras.layers.Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

m = keras.layers.Dropout(dropout)(m)
m = keras.layers.Dense(hidden_dims, activation="relu")(m)
out = keras.layers.Dense(1, activation="sigmoid")(m)


In [ ]:
model = keras.Model(inp, out)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs,
          validation_data=(X_test, y_test), verbose=2)